In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from statistics import mean
from scipy import stats

In [21]:
df = pd.read_pickle("cleaned_df.pkl")

In [22]:
df

,Name,Platform,release_date,summary,meta_score,user_review,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,1998-11-23,as young boy link tricked ganondorf king gerud...,99,9.1,1998,Action,Nintendo,4.10,1.89,1.45,0.16,7.60
1,Tony Hawk's Pro Skater 2,PlayStation,2000-09-20,as most major publishers development efforts s...,98,7.4,2000,Sports,Activision,3.05,1.41,0.02,0.20,4.68
2,Grand Theft Auto IV,PlayStation 3,2008-04-29,"metacritic 2008 ps3 game year also known ""gta ...",98,7.7,2008,Action,Take-Two Interactive,4.76,3.76,0.44,1.62,10.57
3,SoulCalibur,Dreamcast,1999-09-08,this tale souls swords transcending world hist...,98,8.4,1999,Fighting,Namco Bandai Games,0.00,0.00,0.34,0.00,0.34
4,Grand Theft Auto IV,Xbox 360,2008-04-29,metacritic 2008 xbox 360 game year also known ...,98,7.9,2008,Action,Take-Two Interactive,6.76,3.10,0.14,1.03,11.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5289,Pulse Racer,Xbox,2003-01-02,pulse racer takes future racers careen toward ...,24,2.2,2003,Racing,Jaleco,0.01,0.00,0.00,0.00,0.02
5290,Fighter Within,Xbox One,2013-11-19,unleash inner fighter beat friends bare knuckl...,23,2.8,2013,Fighting,Ubisoft,0.11,0.07,0.00,0.02,0.19
5291,Homie Rollerz,DS,2008-03-05,homie rollerz fast paced mayhem laden kart rac...,23,3.0,2008,Racing,Destineer,0.07,0.00,0.00,0.01,0.07
5292,Charlie's Angels,GameCube,2003-07-09,join natalie dylan alex intense adventure base...,23,4.3,2003,Action,Ubisoft,0.01,0.00,0.00,0.00,0.02


## Find words that significantly increase Global Sales

In [51]:
# find all unique words in the summaries and select only that ones that appear >500 times
unique = dict(Counter(" ".join(df["summary"]).split()))
unique_copy = unique.copy()
for key in unique_copy:
    if unique_copy[key] <= 500:
        del unique[key]
len(unique) # how many words show up more than 500 times in the summaries

81

In [52]:
unique

{'players': 3846,
 'game': 3815,
 'power': 657,
 'moves': 505,
 '2': 728,
 'out': 929,
 'make': 519,
 'two': 639,
 'player': 1625,
 'one': 1170,
 'other': 699,
 'before': 545,
 'against': 885,
 'world': 2514,
 'racing': 639,
 'new': 4813,
 'games': 973,
 'first': 917,
 'time': 1342,
 'ever': 618,
 'also': 977,
 'characters': 1083,
 'series': 825,
 'features': 1291,
 'use': 814,
 'battle': 1234,
 'different': 525,
 'control': 1021,
 'play': 1753,
 'online': 853,
 'friends': 610,
 'most': 899,
 'experience': 1525,
 'action': 1440,
 'more': 1976,
 'the': 587,
 'gameplay': 1142,
 'adventure': 669,
 'way': 826,
 'over': 975,
 'through': 1639,
 'enemies': 721,
 'only': 643,
 'both': 503,
 'multiplayer': 793,
 'mode': 1247,
 'each': 928,
 'his': 1603,
 'character': 547,
 'based': 701,
 'race': 558,
 'every': 558,
 'original': 543,
 'using': 534,
 'city': 746,
 'now': 754,
 'unique': 873,
 'own': 845,
 'real': 604,
 'he': 668,
 'set': 679,
 'life': 617,
 'fight': 667,
 'including': 1222,
 'com

In [53]:
# create empty df to fill with whether or not each game's summary contains each word
words = pd.DataFrame(df[["Name","Global_Sales"]],columns = ["Name","Global_Sales"]+list(unique.keys()))
words.head()

,Name,Global_Sales,players,game,power,moves,2,out,make,two,...,modes,such,team,system,evil,gamers,skills,must,variety,man
0,Wii Sports,82.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mario Kart Wii,35.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Wii Sports Resort,33.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,New Super Mario Bros.,30.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,New Super Mario Bros. Wii,28.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# fill data frame with whether the game contains the word (T or F)
for i in range(len(df)):
    for j in list(unique.keys()):
        if j in df["summary"][i]:
            words[j][i] = True
        else:
            words[j][i] = False
words.head()

/var/folders/xx/s138h7zs36sdpnp0qvy3wk2m0000gn/T/ipykernel_51916/397491220.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  words[j][i] = True
/var/folders/xx/s138h7zs36sdpnp0qvy3wk2m0000gn/T/ipykernel_51916/397491220.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  words[j][i] = False


,Name,Global_Sales,players,game,power,moves,2,out,make,two,...,modes,such,team,system,evil,gamers,skills,must,variety,man
0,Wii Sports,82.74,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,Mario Kart Wii,35.82,True,True,True,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
2,Wii Sports Resort,33.00,True,True,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,New Super Mario Bros.,30.01,True,True,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,New Super Mario Bros. Wii,28.62,True,True,False,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,True


In [62]:
# create df that contains the mean user review of games with and without the words
words_aves = pd.DataFrame(columns = ["word","ave_with","ave_without"])
words_aves["word"] = words.columns[2:] # make each word a row
words_aves.head()
for i in list(unique.keys()):
    words_aves.loc[words_aves["word"] == i , "ave_with"] = mean(words.Global_Sales[words[i] == True])
    words_aves.loc[words_aves["word"] == i , "ave_without"] = mean(words.Global_Sales[words[i] == False])
words_aves.head()

,word,ave_with,ave_without
0,players,0.950269,0.794318
1,game,0.870735,0.824278
2,power,0.832115,0.858525
3,moves,0.884316,0.849663
4,2,0.882722,0.838102
5,out,0.955459,0.808215
6,make,1.121997,0.815633
7,two,0.918817,0.843682
8,player,0.913092,0.79274
9,one,0.866844,0.844493


In [70]:
# test for significance - are the average global sales significantly different with and without each word?
# two-sided ttest for each word
# when pvalue small, reject the null hyp that the means are equal
words_aves["pvalue"]=""
for word in words.columns[2:]:
    test = stats.ttest_ind(words["Global_Sales"][words[word]== True], \
                       words["Global_Sales"][words[word]== False])
    words_aves["pvalue"][words_aves["word"]==word] = test.pvalue

In [74]:
words_aves["significant"] = words_aves["pvalue"].map(lambda x: x < .05)
words_aves

,word,ave_with,ave_without,pvalue,significant
0,players,0.950269,0.794318,0.009333,True
1,game,0.870735,0.824278,0.434022,False
2,power,0.832115,0.858525,0.701492,False
3,moves,0.884316,0.849663,0.745809,False
4,2,0.882722,0.838102,0.472674,False
...,...,...,...,...,...
76,gamers,0.71561,0.866962,0.12441,False
77,skills,0.567101,0.882659,0.001365,True
78,must,0.630269,0.884575,0.003576,True
79,variety,0.632661,0.873652,0.018581,True
